In [1]:
from __future__ import annotations
import ast
import re
from dataclasses import dataclass
from typing import List, Dict, Tuple
import pandas as pd
import numpy as np
import os

In [5]:
df = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/final_sorted_samples-images3.csv")
df['assigned_target_label'].value_counts()

assigned_target_label
psoriasis                       460
dermatitis                      324
healthy                         279
systemic lupus erythematosus    111
melanoma                         55
scleroderma                      24
vitiligo                         16
Name: count, dtype: int64

In [26]:
caps = {
    "dermatitis": 208,
    "melanoma": 22,
    "psoriasis": 3,
    "scleroderma": 94,
    "systemic lupus erythematosus": 41,  # or "sle" if that's your label
    "vitiligo": 3,
    "healthy": 279,
}

# SKIP
# df2 = df[df["assigned_target_label"].isin(caps)].copy()

# # keep first N within each label (df order is preserved)
# k = df2.groupby("assigned_target_label").cumcount()
# df_sub = df2[k < df2["assigned_target_label"].map(caps)]
# SKIP 

# FOR
df_sub = (
    df[df["assigned_target_label"].isin(caps)]
      .groupby("assigned_target_label", group_keys=False)
      .apply(lambda g: g.sample(n=min(len(g), caps[g.name]), random_state=42))
).reset_index(drop = True)
# FOR

print(df_sub["assigned_target_label"].value_counts())

assigned_target_label
healthy                         279
dermatitis                      208
systemic lupus erythematosus     41
scleroderma                      24
melanoma                         22
psoriasis                         3
vitiligo                          3
Name: count, dtype: int64


/tmp/ipykernel_170743/1904010232.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min(len(g), caps[g.name]), random_state=42))


In [28]:
df_sub.to_csv("/home/provido/provido/luna/image_data/scin_meta/final_sorted_samples-images4.csv")

### Actual loading and saving

In [30]:
from PIL import Image
import pickle

def load_images_from_list(image_paths):
    images = []
    for p in image_paths:
        try:
            img = Image.open(p).convert("RGB")
            images.append(img)
        except Exception as e:
            print(f"Error loading {p}: {e}")
            images.append(None)
    return images

paths = df_sub["image_path"].tolist()
images = load_images_from_list(paths)

# If you want strict alignment with df, keep None.
# If you want to drop failures:
# images = [im for im in images if im is not None]

with open("/home/provido/provido/luna/image_data/scin_meta/images_sorted.pkl", "wb") as f:
    pickle.dump(images, f, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
images

[<PIL.Image.Image image mode=RGB size=799x1080>,
 <PIL.Image.Image image mode=RGB size=810x1080>,
 <PIL.Image.Image image mode=RGB size=810x1080>,
 <PIL.Image.Image image mode=RGB size=604x796>,
 <PIL.Image.Image image mode=RGB size=810x1065>,
 <PIL.Image.Image image mode=RGB size=654x1074>,
 <PIL.Image.Image image mode=RGB size=642x874>,
 <PIL.Image.Image image mode=RGB size=810x1080>,
 <PIL.Image.Image image mode=RGB size=810x1080>,
 <PIL.Image.Image image mode=RGB size=810x1080>,
 <PIL.Image.Image image mode=RGB size=810x1080>,
 <PIL.Image.Image image mode=RGB size=809x1080>,
 <PIL.Image.Image image mode=RGB size=810x1080>,
 <PIL.Image.Image image mode=RGB size=1080x994>,
 <PIL.Image.Image image mode=RGB size=806x1080>,
 <PIL.Image.Image image mode=RGB size=1080x810>,
 <PIL.Image.Image image mode=RGB size=1080x813>,
 <PIL.Image.Image image mode=RGB size=806x1076>,
 <PIL.Image.Image image mode=RGB size=520x793>,
 <PIL.Image.Image image mode=RGB size=810x1080>,
 <PIL.Image.Image image

### 